In [21]:
import os
import numpy as np
import numpy.ma as ma
import fitsio
import astropy.table

In [5]:
drsuffix = 'dr6-dr7'
LSLGAdir = os.getenv('LSLGA_DIR')
parentfile = os.path.join(LSLGAdir, 'sample', 'large-galaxies-parent.fits')

In [17]:
def read_hyperleda():
    """Read the Hyperleda catalog.
    
    """
    hyperledafile = os.path.join(LSLGAdir, 'sample', 'hyperleda-d25min10.fits')

    leda = astropy.table.Table(fitsio.read(hyperledafile, ext=1))
    print('Read {} objects from {}'.format(len(leda), hyperledafile))

    return leda

In [25]:
def build_parent(cat, nside=128):
    """Identify the galaxies in the nominal LS/DESI footprint."""
    import desimodel.io
    import desimodel.footprint
    
    tiles = desimodel.io.load_tiles(onlydesi=True)
    indesi = desimodel.footprint.is_point_in_desi(tiles, ma.getdata(cat['ra']), 
                                                  ma.getdata(cat['dec']))
    keep = np.where( indesi * np.isfinite(cat['mag']) )[0] # about 13,000 objects have no magnitude...
    
    parent = cat[keep]
    ngal = len(parent)
    
    print('Keeping {} / {} objects in the DESI footprint.'.format(ngal, len(cat)))
    #print('Writing {}'.format(parentfile))
    #parent.write(parentfile, overwrite=True)    
    
    return parent

In [6]:
def write_viewer_file(cat):
    outfile = os.path.join(LSLGAdir, 'sample', 'viewer-sdss-galaxies.fits')
    out = astropy.table.Table()
    out['ra'] = cat['ra']
    out['dec'] = cat['dec']
    out['name'] = cat['galaxy']
    print('Writing {}'.format(outfile))
    out.write(outfile, format='fits', overwrite=True)

In [18]:
leda = read_hyperleda()

Read 2118186 objects from /Users/ioannis/research/projects/LSLGA/sample/hyperleda-d25min10.fits


In [26]:
parent = build_parent(leda)

Keeping 1317271 / 2118186 objects in the DESI footprint.


In [29]:
indx = ['SDSS' in gg for gg in parent['galaxy']] * (parent['d25'] > 1)
print(np.sum(indx))

316


In [30]:
write_viewer_file(parent[indx])

Writing /Users/ioannis/research/projects/LSLGA/sample/viewer-sdss-galaxies.fits


In [34]:
indx = ['SDSS' in gg for gg in parent['galaxy']] * np.isfinite(parent['z']) * (parent['d25'] > 1)
print(np.sum(indx))

161


In [35]:
write_viewer_file(parent[indx])

Writing /Users/ioannis/research/projects/LSLGA/sample/viewer-sdss-galaxies.fits
